In [1]:
#!pip install tqdm              # run this one time to install if you don't have it
#!pip install requests_html     # run this one time to install if you don't have it

import pandas as pd
from requests_html import HTMLSession
import os
import re
from time import sleep
from tqdm.notebook import tqdm # progress bar

session = HTMLSession()

In [2]:
session = HTMLSession() 
url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
r = session.get(url)
r.html.links # gives all the relative links  - you cant click on them 
r.html.absolute_links # makes this clickable immediately- you can clink on all of them 

# get the tabe 
r.html.find('table')  #[0]
table = r.html.find('#constituents') [0] # id so use # sign 
table.text # just text 
table.html # print html 

# blunt force the table! 
table.absolute_links # extracted all urls from the table. still too many links but can "dumb filter"
 #pd.read_html(table.html)- only gives html tho 
r.html.links # gives all the relative links  - you cant click on them 
r.html.absolute_links # makes this clickable immediately- you can clink on all of them 

# get the tabe 
r.html.find('table')  #[0]
table = r.html.find('#constituents') [0] # id so use # sign 
table.text # just text 
table.html # print html 

# blunt force the table! 
table.absolute_links # extracted all urls from the table. still too many links but can "dumb filter"
 #pd.read_html(table.html)- only gives html tho 
table.find('tr')[0].text # header...we dont want that!
table.find('tr')[1].text

table_rows = table.find('tr')

colinks = []
for row in table_rows[1:]:
    print(row.find('td')[1].absolute_links) # got swuigly bracket so a set 
 # colink append  
    a_link = list(row.find('td')[1].absolute_links)[0] 
    colinks.append(a_link)
len(colinks)

{'https://en.m.wikipedia.org/wiki/3M'}
{'https://en.m.wikipedia.org/wiki/Abbott_Laboratories'}
{'https://en.m.wikipedia.org/wiki/AbbVie_Inc.'}
{'https://en.m.wikipedia.org/wiki/Abiomed'}
{'https://en.m.wikipedia.org/wiki/Accenture_plc'}
{'https://en.m.wikipedia.org/wiki/Activision_Blizzard'}
{'https://en.m.wikipedia.org/wiki/Adobe_Inc.'}
{'https://en.m.wikipedia.org/wiki/Advanced_Micro_Devices_Inc'}
{'https://en.m.wikipedia.org/wiki/Advance_Auto_Parts'}
{'https://en.m.wikipedia.org/wiki/AES_Corp'}
{'https://en.m.wikipedia.org/wiki/AFLAC_Inc'}
{'https://en.m.wikipedia.org/wiki/Agilent_Technologies_Inc'}
{'https://en.m.wikipedia.org/wiki/Air_Products_%26_Chemicals_Inc'}
{'https://en.m.wikipedia.org/wiki/Akamai_Technologies_Inc'}
{'https://en.m.wikipedia.org/wiki/Alaska_Air_Group_Inc'}
{'https://en.m.wikipedia.org/wiki/Albemarle_Corp'}
{'https://en.m.wikipedia.org/wiki/Alexandria_Real_Estate_Equities'}
{'https://en.m.wikipedia.org/wiki/Alexion_Pharmaceuticals'}
{'https://en.m.wikipedia.or

{'https://en.m.wikipedia.org/wiki/Marsh_%26_McLennan'}
{'https://en.m.wikipedia.org/wiki/Martin_Marietta_Materials'}
{'https://en.m.wikipedia.org/wiki/Masco_Corp.'}
{'https://en.m.wikipedia.org/wiki/Mastercard_Inc.'}
{'https://en.m.wikipedia.org/wiki/McCormick_%26_Co.'}
{'https://en.m.wikipedia.org/wiki/Maxim_Integrated'}
{'https://en.m.wikipedia.org/wiki/McDonald%27s_Corp.'}
{'https://en.m.wikipedia.org/wiki/McKesson_Corp.'}
{'https://en.m.wikipedia.org/wiki/Medtronic_plc'}
{'https://en.m.wikipedia.org/wiki/Merck_%26_Co.'}
{'https://en.m.wikipedia.org/wiki/MetLife_Inc.'}
{'https://en.m.wikipedia.org/wiki/Mettler_Toledo'}
{'https://en.m.wikipedia.org/wiki/MGM_Resorts_International'}
{'https://en.m.wikipedia.org/wiki/Microchip_Technology'}
{'https://en.m.wikipedia.org/wiki/Micron_Technology'}
{'https://en.m.wikipedia.org/wiki/Microsoft_Corp.'}
{'https://en.m.wikipedia.org/wiki/Mid-America_Apartments'}
{'https://en.m.wikipedia.org/wiki/Mohawk_Industries'}
{'https://en.m.wikipedia.org/wik

505

In [3]:
#### SECTION 1: get the urls to crawl (or data that can build the urls)

if not os.path.exists('input/sp500_urls.csv'):

    # open page
    session = HTMLSession() 
    url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
    r = session.get(url)

    # find the links
    table = r.html.find('#constituents')[0]  # reduce to the table
    table_rows = table.find('tr')[1:]
    colinks = []
    for row in table_rows:
        a_url =   list(row.find('td')[1].absolute_links)[0]    
        colinks.append(a_url)
        
          # save list to csv    
if not os.path.exists('input/'):
    os.makedirs('input/')
    pd.DataFrame(colinks).to_csv('input/sp500_urls.csv',index=False,header=False)
    
else:
    colinks = pd.read_csv('input/sp500_urls.csv',names=['firm']) ['firm'].to_list()
    
#### SECTION 2: crawl the urls

# for url in colinks[:10]:
for url in tqdm(colinks,total=len(colinks)):
    
    folder = 'wiki_html/'
    if not os.path.exists('wiki_html/'):
        os.makedirs('wiki_html/')
    
    name = re.sub( r'\W+','',     url.split('/')[-1]    )
    filepath = folder + name + '.txt'
        
    if not os.path.exists(filepath):
        
        try:
            r = session.get(url)
        except:
            print('sad face failure:',url)
        else:   
            with open(filepath,'w',encoding='utf8') as f:
                f.write(r.text)
            sleep(2) # be nice to server    